In [1]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages
Processing /kaggle/input/pip-packages-icr/pip-packages/tabpfn-0.1.9-py3-none-any.whl


In [2]:
import time
import numpy as np
import pandas as pd
import warnings
def ignore_warn(*args,**kwargs):
    pass
warnings.warn = ignore_warn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import f1_score
import xgboost as xgboost
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, HistGradientBoostingClassifier, BaggingClassifier, StackingClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting  
from sklearn.model_selection import cross_val_predict
from sklearn.impute import KNNImputer
from tabpfn import TabPFNClassifier

In [3]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
final = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

In [4]:
encoder = LabelEncoder()
train.loc[:, 'EJ'] = encoder.fit_transform(train.loc[:, 'EJ'])
test.loc[:, 'EJ'] = encoder.fit_transform(test.loc[:, 'EJ'])

In [5]:
#column_names = train.columns
#imputer = KNNImputer(n_neighbors = 10)
#train_no_id = imputer.fit_transform(train.drop(['Id'], axis = 1))
#train_no_id = pd.DataFrame(train_no_id)
#train = pd.concat([train['Id'], train_no_id], axis = 1)
#train.columns = column_names

In [6]:
#train.isna().sum().sort_values(ascending = False)

In [7]:
# taken directly from https://www.kaggle.com/code/chensilin/icr-eda-lightgbm-xgboost-optuna/input
seed = 617
zero, one = np.bincount(train.loc[:, 'Class'])
one_df = train.iloc[(train.loc[:, 'Class'] == 1).tolist(), :] 
zero_df = train.iloc[(train.loc[:, 'Class'] == 0).tolist(), :]
zero_df = zero_df.sample(n=one, random_state=seed)
oversampled_df = pd.concat([train.iloc[(train.loc[:, 'Class'] == 0).tolist(), :], one_df, one_df, one_df, one_df])
oversampled_df = oversampled_df.sample(frac=1, random_state=seed)

In [8]:
# taken directly from https://www.kaggle.com/code/chensilin/icr-eda-lightgbm-xgboost-optuna/input
def balanced_log_loss(y_true, y_pred):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    p_1 = np.clip(y_pred[:, 1], 1e-15, 1-1e-15)
    p_0 = 1 - p_1
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    balanced_log_loss = (w_0 * log_loss_0 + w_1 * log_loss_1) / 2
    return balanced_log_loss

In [9]:
# taken directly from https://www.kaggle.com/code/chensilin/icr-eda-lightgbm-xgboost-optuna/input
n_folds = 10
def CV(model, data, loss_function):
    skf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    kfold = skf.split(data.iloc[:, 1:-1], data.iloc[:, -1])
    losses = []
    for (train_id, val_id) in kfold:
        x_train = data.iloc[train_id, 1:-1]
        y_train = data.iloc[train_id, -1]
        x_val = data.iloc[val_id, 1:-1]
        y_val = data.iloc[val_id, -1]
        model.fit(x_train, y_train)
        pred_val = model.predict_proba(x_val)
        loss = loss_function(y_val, pred_val)
        losses.append(loss)
    return np.sum(losses) / n_folds

In [10]:
X_train = oversampled_df.drop(columns=['Class', 'Id'])
y_train = oversampled_df['Class']

In [11]:
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [12]:
def xgb(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
    }
    model = xgb.XGBClassifier(**param, random_state = seed)
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [13]:
def lgbm(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'num_leaves' : trial.suggest_int('num_leaves', 10, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 50),
    }
    model = lgb.LGBMClassifier(**param, random_state = seed)
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [14]:
def catboost(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 150, 10),
        'reg_lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 10),
    }
    model = CatBoostClassifier(**param, random_seed=seed, verbose=False)
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [15]:
def hgbc(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_iter': trial.suggest_int('max_iter', 50, 200, 10),
    }
    model = HistGradientBoostingClassifier(**param, random_state=seed)
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [16]:
# def bagged_dt(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 400, 600, 20),
#         'max_samples': trial.suggest_float('max_samples', 1e-2, 1.0),
#         'max_features': trial.suggest_float('max_features', 1e-2, 1.0),
#         'bootstrap': trial.suggest_categorical('bootstrap', [False, True]),
#         'bootstrap_features': trial.suggest_categorical('bootstrap_features', [False, True]),
#     }
#     model = BaggingClassifier(**param, base_estimator=DecisionTreeClassifier(), random_state=seed)
#     score = CV(model, oversampled_df, balanced_log_loss)
#     return score

In [17]:
# def rf(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 300, 500, 20),
#         'max_depth': trial.suggest_int('max_depth', 5, 25),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
#         'max_features': trial.suggest_int('max_features', 1, 56),
#         'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
#     }
#     model = RandomForestClassifier(**param, random_state=seed)
#     score = CV(model, oversampled_df, balanced_log_loss)
#     return score

In [18]:
# def adaboost(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 400, 500, 20),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 1),
#         'base_estimator': DecisionTreeClassifier(max_depth=trial.suggest_int('max_depth', 1, 20)),
#     }
#     model = AdaBoostClassifier(**param, random_state=seed)
#     score = CV(model, oversampled_df, balanced_log_loss)
#     return score

In [19]:
# def gradient(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 10, 500, 20),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 1.0),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
#         'subsample': trial.suggest_float('subsample', 0.3, 1.0),
#     }
#     model = GradientBoostingClassifier(**param, random_state=seed)
#     score = CV(model, oversampled_df, balanced_log_loss)
#     return score

In [20]:
# def tabpfn(trial):
#     param = {
#         'N_ensemble_configurations': trial.suggest_int('N_ensemble_configurations', 20, 70),
#     }
#     model = TabPFNClassifier(**param)
#     score = CV(model, oversampled_df, balanced_log_loss)
#     return score

In [1]:
cv_scores = []
for i in range(20, 70):
    cv_scores.append(CV(TabPFNClassifier(N_ensemble_configurations=i).fit(X_train, y_train), oversampled_df, balanced_log_loss))
    print("Finished scoring iteration: ", i)
print(cv_scores)

NameError: name 'CV' is not defined

In [21]:
# CV(TabPFNClassifier(N_ensemble_configurations=24).fit(X_train, y_train), oversampled_df, balanced_log_loss)
# # for each test performance on 1s and 0s

In [ ]:
#add way to do rule-based and adaptive ensembling
#see what others did

In [22]:
# start = time.time()

# study1 = optuna.create_study(direction='minimize', study_name="TabPFN")
# n_trials = 20
# study1.optimize(tabpfn, n_trials=n_trials)
# print('Best trial:', study1.best_trial.params)
# print('Best values:', study1.best_value)

# end = time.time()
# print('It has taken {:.5f} seconds to search for the best Hyperparameter'.format(end-start))

In [23]:
xgb_model = xgboost.XGBClassifier(n_estimators=200, reg_lambda=0.15325900166549988, reg_alpha=0.0044700650580230235, 
                              colsample_bytree=0.3, subsample=1.0, learning_rate=0.09975673376458177, 
                              max_depth=11, min_child_weight=2, random_state=seed).fit(X_train, y_train)
xgb = xgb_model.predict_proba(test.iloc[:, 1:])

In [24]:
lgbm_model = lgb.LGBMClassifier(n_estimators=200, reg_alpha=0.0016725623110267532, reg_lambda=0.0038043774323061946, 
                                 colsample_bytree=0.3, subsample=0.4, learning_rate=0.09367295744238123, max_depth=11, 
                                 num_leaves=50, min_child_samples=26, random_state=seed).fit(X_train, y_train)
lgbm = lgbm_model.predict_proba(test.iloc[:, 1:])

In [25]:
cat_model = CatBoostClassifier(n_estimators=70, reg_lambda=0.01606738047167, colsample_bylevel=0.3, 
                              subsample=0.7, learning_rate=0.0865881098465479, 
                              max_depth=9, one_hot_max_size=10, random_state=seed, verbose=False).fit(X_train, y_train)
cat = cat_model.predict_proba(test.iloc[:, 1:])

In [26]:
hgbc_model = HistGradientBoostingClassifier(max_iter=170, max_depth=4, min_samples_leaf=9,
                                         learning_rate=0.17193627413211837, random_state=seed).fit(X_train, y_train)
hgbc = hgbc_model.predict_proba(test.iloc[:, 1:])

In [27]:
# dt_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=500, max_samples=0.994711990802652,
#                            max_features=0.7844039008030275, bootstrap=False, bootstrap_features=True, random_state=seed).fit(X_train, y_train)
# dt = dt_model.predict_proba(test.iloc[:, 1:])

In [28]:
# rf_model = RandomForestClassifier(n_estimators=420, max_depth=13, min_samples_split=3, 
#                                   min_samples_leaf=1, max_features=9, bootstrap=False, random_state=seed).fit(X_train, y_train)
# rf = rf_model.predict_proba(test.iloc[:, 1:])

In [29]:
# ada_model = AdaBoostClassifier(n_estimators=420, learning_rate=0.6794771047521856, 
#                                base_estimator=DecisionTreeClassifier(max_depth=4), random_state=seed).fit(X_train, y_train)
# ada = ada_model.predict_proba(test.iloc[:, 1:])

In [30]:
# gradient_model = GradientBoostingClassifier(n_estimators=170, learning_rate=0.05893342084840253, max_depth=6, 
#                                             min_samples_split=16, min_samples_leaf=11, subsample=0.9112739951878389, random_state=seed).fit(X_train, y_train)
# gradient = gradient_model.predict_proba(test.iloc[:, 1:])

In [31]:
def catboost_meta(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 150, 10),
        'reg_lambda': trial.suggest_loguniform('lambda', 1e-3, 0.1),
        'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 10),
    }
    model = StackingClassifier(
        estimators=[("catboost", cat_model), ("hist_gradient_boosting", hgbc_model),
                    ("lgbm", lgbm_model), ("xgboost", xgb_model)],
        final_estimator=CatBoostClassifier(**param, random_state=seed, verbose=False)
    )
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [32]:
def xgb_meta(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 0.1),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 0.1),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
    }
    model = StackingClassifier(
        estimators=[("catboost", cat_model), ("hist_gradient_boosting", hgbc_model),
                    ("lgbm", lgbm_model), ("xgboost", xgb_model)],
        final_estimator=xgboost.XGBClassifier(**param, random_state=seed)
    )
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [33]:
def hgbc_meta(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_iter': trial.suggest_int('max_iter', 50, 200, 10),
    }
    model = StackingClassifier(
        estimators=[("catboost", cat_model), ("hist_gradient_boosting", hgbc_model),
                    ("lgbm", lgbm_model), ("xgboost", xgb_model)],
        final_estimator=HistGradientBoostingClassifier(**param, random_state=seed)
    )
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [34]:
def lgbm_meta(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 0.1),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 0.1),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'num_leaves' : trial.suggest_int('num_leaves', 10, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 50),
    }
    model = StackingClassifier(
        estimators=[("catboost", cat_model), ("hist_gradient_boosting", hgbc_model),
                    ("lgbm", lgbm_model), ("xgboost", xgb_model)],
        final_estimator=lgb.LGBMClassifier(**param, random_state=seed)
    )
    score = CV(model, oversampled_df, balanced_log_loss)
    return score

In [35]:
stacking_cat_model = CatBoostClassifier(n_estimators=150, reg_lambda=0.05055956136270572, colsample_bylevel=0.6, 
                                        subsample=0.5, learning_rate=0.08699165501504001, max_depth=7, 
                                        one_hot_max_size=8, random_state=seed, verbose=False).fit(X_train, y_train)
stacking_cat = stacking_cat_model.predict_proba(test.iloc[:, 1:])

In [36]:
stacking_xgb_model = xgboost.XGBClassifier(n_estimators=160, reg_lambda=0.030554982480056614, alpha=0.022993963306149747, 
                                           colsample_bytree=0.4, subsample=0.6, learning_rate=0.08378145372235492, 
                                           max_depth=17, min_child_weight=1, random_state=seed).fit(X_train, y_train)
stacking_xgb = stacking_xgb_model.predict_proba(test.iloc[:, 1:])

In [37]:
stacking_lgbm_model = lgb.LGBMClassifier(n_estimators=130, reg_alpha=0.017987440901161444, reg_lambda=0.0010110144342120994, 
                                colsample_bytree=0.8, subsample=0.5, learning_rate=0.08786840365732179, 
                                max_depth=5, num_leaves=10, min_child_samples=48, random_state=seed).fit(X_train, y_train)
stacking_lgbm = stacking_lgbm_model.predict_proba(test.iloc[:, 1:])

In [38]:
stacking_hgbc_model = HistGradientBoostingClassifier(learning_rate=0.9112141545526848, max_depth=5, min_samples_leaf=18, 
                                                     max_iter=160, random_state=seed).fit(X_train, y_train)
stacking_hgbc = stacking_hgbc_model.predict_proba(test.iloc[:, 1:])

In [39]:
from sklearn.metrics import get_scorer_names

In [40]:
print('CatBoostClassifier CV: ', CV(cat_model, oversampled_df, balanced_log_loss))

CatBoostClassifier CV:  0.025026638813281182


In [41]:
print('XGB Classifier CV: ', CV(xgb_model, oversampled_df, balanced_log_loss))

XGB Classifier CV:  0.035177945263509695


In [42]:
print('HGBC Classifier CV: ', CV(hgbc_model, oversampled_df, balanced_log_loss))

HGBC Classifier CV:  0.024601263961530347


In [43]:
print('LGBM Classifier CV: ', CV(lgbm_model, oversampled_df, balanced_log_loss))

LGBM Classifier CV:  0.021615876584382923


In [44]:
print('CatBoostClassifier Stacking CV: ', CV(stacking_cat_model, oversampled_df, balanced_log_loss))

CatBoostClassifier Stacking CV:  0.04163203397643795


In [45]:
print('XGB Classifier Stacking CV: ', CV(stacking_xgb_model, oversampled_df, balanced_log_loss))

XGB Classifier Stacking CV:  0.04439938244132395


In [46]:
print('LGBM Classifier Stacking CV: ', CV(stacking_lgbm_model, oversampled_df, balanced_log_loss))

LGBM Classifier Stacking CV:  0.03831973024046782


In [47]:
print('HGBC Classifier Stacking CV: ', CV(stacking_hgbc_model, oversampled_df, balanced_log_loss))

HGBC Classifier Stacking CV:  0.051569306270565204


In [48]:
def round_up_down(num):
    return 1 if num >= 0.5 else 0

In [49]:
#final.iloc[:, 1], final.iloc[:, -1] = stacking_cat_model.predict_proba(test.iloc[:, 1:])[:, 0], stacking_cat_model.predict_proba(test.iloc[:, 1:])[:, 1]
final.iloc[:, 1] = (lgbm[:, 0] + xgb[:, 0])/2
final.iloc[:, -1] = (lgbm[:, 1] + xgb[:, 1])/2
final.iloc[:, 1:] = final.iloc[:, 1:].applymap(round_up_down)
final.to_csv('submission.csv', index=False)
submission = pd.read_csv('submission.csv')
submission

,Id,class_0,class_1
0,00eed32682bb,1,0
1,010ebe33f668,1,0
2,02fa521e1838,1,0
3,040e15f562a2,1,0
4,046e85c7cc7f,1,0
